In [11]:
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision
import pickle

In [4]:
device = torch.device('cuda')
device

device(type='cuda')

# Load Data

In [86]:
from torch.utils.data import Dataset

class VariableLenDataset(Dataset):
    def __init__(self, data, targets):
        self.X = [torch.tensor(x, dtype=torch.float32) for x in data]
        self.y = torch.tensor(targets, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [7]:
train_transform = transforms.Compose(
    [transforms.ToTensor()])

In [13]:
with open("data/train.pkl", "rb") as train_file:
    dataset = pickle.load(train_file)

In [57]:
df = pd.DataFrame(dataset)
df

,0,1
0,"[-1.0, -1.0, -1.0, -1.0, 144.0, 144.0, 144.0, ...",0
1,"[-1.0, -1.0, 144.0, 144.0, 34.0, 0.0, 4.0, 2.0...",0
2,"[66.0, 100.0, 148.0, 148.0, 146.0, 64.0, 146.0...",0
3,"[147.0, 65.0, 64.0, 146.0, 8.0, 50.0, 160.0, 5...",0
4,"[144.0, 144.0, 144.0, 144.0, 144.0, 64.0, 64.0...",0
...,...,...
2934,"[144.0, 144.0, 144.0, 66.0, 82.0, 132.0, 8.0, ...",4
2935,"[32.0, 33.0, 162.0, 121.0, 68.0, 68.0, 120.0, ...",4
2936,"[33.0, 33.0, 33.0, 33.0, 78.0, 2.0, 68.0, 34.0...",4
2937,"[96.0, 96.0, 96.0, 96.0, 176.0, 176.0, 176.0, ...",4


In [87]:
train_dataset = VariableLenDataset(df[0].values, df[1].values)

In [88]:
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

pad = 0

def pad_collate(batch, pad_value=0):
    xx, yy = zip(*batch)
    x_lens = [len(x) for x in xx]
    y_lens = [len(y) for y in yy]

    xx_pad = pad_sequence(xx, batch_first=True, padding_value=pad_value)
    yy_pad = pad_sequence(yy, batch_first=True, padding_value=pad_value)

    return xx_pad, yy_pad, x_lens, y_lens

In [89]:
train_loader = DataLoader(train_dataset, batch_size=50, shuffle=True, collate_fn=pad_collate)

NameError: name 'train_set' is not defined

# Model

In [ ]:
class ComposerClasifier(nn.Module):

    def __init__(self, input_size, hidden_size, num_layers, out_size, bidirectional = False):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        if bidirectional:
            self.bidirectional = 2
        else:
            self.bidirectional = 1
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, bidirectional=bidirectional, dropout=0.4)
        self.fc = nn.Linear(hidden_size*90*self.bidirectional, out_size)

    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers*self.bidirectional , batch_size, self.hidden_size)
        state = torch.zeros(self.num_layers*self.bidirectional , batch_size, self.hidden_size)
        return hidden, state

    def forward(self, x, hidden):
        x = torch.transpose(x,0,1)
        all_outputs, hidden = self.lstm(x, hidden)
        all_outputs = torch.transpose(all_outputs,0,1)
        out = torch.flatten(all_outputs,1)
        x = self.fc(out)
        return x, hidden


In [ ]:
model = ComposerClasifier(1,5,2,16).to(device)
model